In [7]:
#1. Cree un RDD llamado lenguajes que contenga los siguientes lenguajes de programación: Python, R, C, 
#   Scala, Rugby y SQL.

#       A) Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes de programación 
#          estén en mayúsculas.

import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName('Cap3').getOrCreate()

sc = spark.sparkContext
lenguajes = sc.parallelize(["Python", "R", "C", "Scala", "Rugby", "SQL"])

lenguajes_mayusculas = lenguajes.map(lambda x: x.upper())
lenguajes_mayusculas.collect()

['PYTHON', 'R', 'C', 'SCALA', 'RUGBY', 'SQL']

In [9]:
#       B) Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes de programación 
#          estén en minúsculas.
lenguajes_minusculas = lenguajes.map(lambda x: x.lower())
lenguajes_minusculas.collect()



['python', 'r', 'c', 'scala', 'rugby', 'sql']

In [10]:
#       C) Cree un nuevo RDD que solo contenga aquellos lenguajes de 
#         programación que comiencen con la letra R.
lenguajes_con_R = lenguajes.filter(lambda x: x.startswith("R"))
lenguajes_con_R.collect()

['R', 'Rugby']

In [12]:
#2. Cree un RDD llamado pares que contenga los números pares existentes en el intervalo [20;30].
pares = sc.parallelize([x for x in range(20, 31) if x % 2 == 0])
pares.collect()

[20, 22, 24, 26, 28, 30]

In [14]:
#       A) Cree el RDD llamado sqrt, este debe contener la raíz cuadrada de los elementos que componen 
#         el RDD pares.
import math

sqrt = pares.map(lambda x: math.sqrt(x))
sqrt.collect()


[4.47213595499958,
 4.69041575982343,
 4.898979485566356,
 5.0990195135927845,
 5.291502622129181,
 5.477225575051661]

In [15]:
#       B) Obtenga una lista compuesta por los números pares en el intervalo 
#         [20;30] y sus respectivas raíces cuadradas. Un ejemplo del resultado 
#         deseado para el intervalo [50;60] sería la lista [50, 7.0710678118654755, 
#         52, 7.211102550927978, 54, 7.3484692283495345, 56, 7.483314773547883, 58, 7.615773105863909, 
#         60, 7.745966692414834].
pares_y_sqrt = pares.flatMap(lambda x: [x, math.sqrt(x)])
pares_y_sqrt.collect()

[20,
 4.47213595499958,
 22,
 4.69041575982343,
 24,
 4.898979485566356,
 26,
 5.0990195135927845,
 28,
 5.291502622129181,
 30,
 5.477225575051661]

In [16]:
#       C) Eleve el número de particiones del RDD sqrt a 20.
sqrt_reparticionado = sqrt.repartition(20)
sqrt_reparticionado.collect()

[4.898979485566356,
 4.69041575982343,
 4.47213595499958,
 5.0990195135927845,
 5.291502622129181,
 5.477225575051661]

In [17]:
#       D) Si tuviera que disminuir el número de particiones luego de haberlo establecido en 20, 
#         ¿qué función utilizaría para hacer más eficiente su código? : Se usa   la función coalesce, que es más eficiente que repartition 
sqrt_reduccion_particiones = sqrt_reparticionado.coalesce(5)
sqrt_reduccion_particiones.collect()


[4.898979485566356,
 5.477225575051661,
 5.0990195135927845,
 4.47213595499958,
 5.291502622129181,
 4.69041575982343]

In [42]:
#3. Cree un RDD del tipo clave valor a partir de la lectura del txt de transacciones. Tenga en cuenta 
#   que deberá procesar el RDD leído para obtener el resultado solicitado. Supongamos que el RDD 
#   resultante de tipo clave valor refleja las transacciones realizadas por número de cuentas. Obtenga 
#   el monto total por cada cuenta (Suma).
#   Tip: Cree su propia función para procesar el RDD leído.


rdd_transacciones= sc.textFile('./transacciones-Leccion-3-1.txt')
rdd_transacciones.collect()


['(1001, 52.3)',
 '(1005, 20.8)',
 '(1001, 10.1)',
 '(1004, 52.7)',
 '(1005, 20.7)',
 '(1002, 85.3)',
 '(1004, 20.9)']

In [43]:
# Función para procesar cada línea y convertirla en un par clave-valor
def procesar_linea(linea):
    # Eliminar paréntesis y espacios
    linea = linea.strip().replace('(', '').replace(')', '')
    # Dividir por coma
    cuenta, monto = linea.split(',')
    # Convertir a entero y float respectivamente
    return (int(cuenta), float(monto))

# Transformar el RDD en un RDD de clave-valor
rdd_kv = rdd_transacciones.map(procesar_linea)

# Sumar los montos por cada cuenta
resultado = rdd_kv.reduceByKey(lambda a, b: a + b)

# Mostrar los resultados
for cuenta, total in resultado.collect():
    print(f"Cuenta {cuenta}: Total {total}")


Cuenta 1004: Total 73.6
Cuenta 1002: Total 85.3
Cuenta 1001: Total 62.4
Cuenta 1005: Total 41.5
